# RAG

## Requirements

In [1]:
%%capture
!pip install transformers accelerate bitsandbytes langchain langchain-community sentence-transformers faiss-gpu pandas gdown

## Dataset

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI
From (redirected): https://drive.google.com/uc?id=1Lq2zVJlN_B4kUAu4VafQ4jXMIQiAR9vI&confirm=t&uuid=11358b2d-4211-48ba-86cd-ed013eac91df
To: /content/IMDB_crawled.json
100% 292M/292M [00:02<00:00, 126MB/s]


## Config

In [3]:
class Config:
    EMBEDDING_MODEL_NAME="thenlper/gte-base"
    LLM_MODEL_NAME="HuggingFaceH4/zephyr-7b-beta"
    K = 5 # top K retrieval

## Preprocessing

In [4]:
import pandas as pd

df = pd.read_json('IMDB_crawled.json')
df.head()

,id,title,first_page_summary,release_year,mpaa,budget,gross_worldwide,rating,directors,writers,stars,related_links,languages,countries_of_origin,summaries,synposis,reviews,genres
0,tt0071562,The Godfather Part II,The early life and career of Vito Corleone in ...,1974,R,"$13,000,000 (estimated)","$47,962,683",9.0,[Francis Ford Coppola],None,"[Al Pacino, Robert De Niro, Robert Duvall]",[https://imdb.com/title/tt0068646/?ref_=tt_sim...,"[English, Italian, Spanish, Latin, Sicilian]",[United States],[The early life and career of Vito Corleone in...,[The Godfather Part II presents two parallel s...,"[[Coppola's masterpiece is rivaled only by ""Th...","[Crime, Drama]"
1,tt0120737,The Lord of the Rings: The Fellowship of the Ring,A meek Hobbit from the Shire and eight compani...,2001,PG-13,"$93,000,000 (estimated)","$884,041,698",8.9,[Peter Jackson],None,"[Elijah Wood, Ian McKellen, Orlando Bloom]",[https://imdb.com/title/tt0167261/?ref_=tt_sim...,"[English, Sindarin]","[New Zealand, United States]",[A meek Hobbit from the Shire and eight compan...,[Galadriel (Cate Blanchett) (The Elven co-rule...,"[[Here is one film that lived up to its hype, ...","[Action, Adventure, Drama]"
2,tt0110912,Pulp Fiction,"The lives of two mob hitmen, a boxer, a gangst...",1994,R,"$8,000,000 (estimated)","$213,928,762",8.9,[Quentin Tarantino],None,"[John Travolta, Uma Thurman, Samuel L. Jackson]",[https://imdb.com/title/tt0137523/?ref_=tt_sim...,"[English, Spanish, French]",[United States],"[The lives of two mob hitmen, a boxer, a gangs...",[Narrative structure\nPulp Fiction's narrative...,[[I like the bit with the cheeseburger. It mak...,"[Crime, Drama]"
3,tt0068646,The Godfather,The aging patriarch of an organized crime dyna...,1972,R,"$6,000,000 (estimated)","$250,342,030",9.2,[Francis Ford Coppola],None,"[Marlon Brando, Al Pacino, James Caan]",[https://imdb.com/title/tt0071562/?ref_=tt_sim...,"[English, Italian, Latin]",[United States],[The aging patriarch of an organized crime dyn...,"[In late summer 1945, guests are gathered for ...",[['The Godfather' is the pinnacle of flawless ...,"[Crime, Drama]"
4,tt0111161,The Shawshank Redemption,"Over the course of several years, two convicts...",1994,R,"$25,000,000 (estimated)","$28,904,232",9.3,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[https://imdb.com/title/tt0468569/?ref_=tt_sim...,[English],[United States],"[Over the course of several years, two convict...","[In 1947, Andy Dufresne (Tim Robbins), a banke...",[[The Shawshank Redemption is written and dire...,[Drama]


In [5]:
import os

os.makedirs('data', exist_ok=True)

# preprocess your data and only store the needed data as the context window for embedding model is limited

df = df[['title', 'genres', 'rating',  'first_page_summary','summaries']]
df["summaries"] = df["summaries"].apply(lambda x: ' '.join(x) if x else None)


df.to_csv('data/imdb.csv', index=False)

## Vectorizer

load the CSV file and vectorize the rows using HuggingFaceEmbeddings.
Store the results using FAISS vectorstore.
Save the vectorestore in a pickle file for future usages.

In [10]:
import pickle

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain.vectorstores.faiss import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

# load the csv
loader = CSVLoader("data/imdb.csv")
data = loader.load()
print(type(data[0]))


# load the embeddings model

embeddings_model = HuggingFaceEmbeddings(model_name = Config.EMBEDDING_MODEL_NAME)

# save embed the documents using the model in a vectorstore

vectorstore = FAISS.from_documents(data, embeddings_model, distance_strategy=DistanceStrategy.COSINE)

with open("data/vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

<class 'langchain_core.documents.base.Document'>


load the vectorstore as a retriever.

In [11]:
with open("data/vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# load the retriever from the vectorstore
retriever = vectorstore.as_retriever(k=Config.K)

## LLM

load the quantized LLM.

In [12]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# load the quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,

)

model = AutoModelForCausalLM.from_pretrained(Config.LLM_MODEL_NAME, quantization_config=bnb_config, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(Config.LLM_MODEL_NAME)

# init the pipeline
READER_LLM = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
)

llm = HuggingFacePipeline(
    pipeline=READER_LLM,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


initialize the prompt template for the query chain. query chain is used to get a query from the chat history. you may change the prompt as you like to get better results.

In [22]:
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser

class LoggerStrOutputParser(StrOutputParser):
    def parse(self, text: str) -> str:
        # process the LLM output
        print(f"text:\n{text}\nEND")
        return text.split('|>')[-1]

query_transform_prompt = PromptTemplate(
    input_variables=["messages"],
    template="""<|system|>You are a helpful assistant.
{messages}
<|user|>
give me the search query about the above conversation.
<|assistant|>"""
)

# init the query chain
query_transforming_retriever_chain = query_transform_prompt | llm | LoggerStrOutputParser()

initialize the main retrieval chain that gives the resulting documents to LLM and gets the output back.

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate(
    input_variables=["context", "messages"],
    template="""<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

{context}
-----------------
{messages}
<|assistant|>""")

# init the retriver chain
def format_docs(docs):
    return "\n\n############\n".join(doc.page_content for doc in docs)

retriever_chain = (
    {"context": retriever| format_docs , "messages": RunnablePassthrough()}
    | prompt
    | llm
    | LoggerStrOutputParser()
)

write the conversation helper class for easier testing.

In [28]:
class Conversation:
    def __init__(self):
        self.messages = []

    def add_assistant_message(self, message):
        self.messages.append(('assistant', message))

    def add_user_message(self, message):
        self.messages.append(('user', message))

    def get_messages(self):
        # concatenate the messages with the roles in the instruction format
        ans = "\n\n".join([f"<|{role}|>{message}" for role, message in self.messages])
        return ans

    def chat(self, message):
        self.add_user_message(message)
        messages = self.get_messages()
        # invoke the chain
        query = query_transforming_retriever_chain.invoke(messages)

        response = retriever_chain.invoke(query)
        self.add_assistant_message(response)
        return response

## Test

talk with the RAG to see how good it performs.

In [29]:
c = Conversation()
A = c.chat('give me a cool gangster movie')
print(A)

text:
<|system|>You are a helpful assistant.
<|user|>give me a cool gangster movie
<|user|>
give me the search query about the above conversation.
<|assistant|>
"Recommend a cool gangster movie based on a conversation between a user and an assistant discussing movie suggestions."
END
text:
<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

title: Scarface
genres: ['Comedy', 'Crime', 'Drama']
rating: 
first_page_summary: 
summaries: 

############
title: Goodfellas 2
genres: ['Short', 'Comedy']
rating: 5.6
first_page_summary: Two gangsters meet and fall madly in love, and have a little bit of gay romance but their Italian brothers won't let that one slide so they all hit the gritty hop on the Fortnite and win som... Read all
summaries: Two gangsters meet and fall madly in love, and have a little bit of gay romance but their Italian brothers won't let that one slide so they all hit the gritty hop on the Fortnite and win some dubs

############
title: Jigar

In [30]:
A = c.chat('give me a newer one')
print(A)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


text:
<|system|>You are a helpful assistant.
<|user|>give me a cool gangster movie

<|assistant|>
Based on your preferences and the movie options provided, I would highly recommend "Goodfellas 2" for a cool gangster movie. While it may not have the same reputation as the original "Goodfellas," this sequel combines elements of comedy and romance in a unique and entertaining way. The movie follows the story of two gangsters falling in love, but their Italian brothers' disapproval leads to chaos and action-packed scenes. With a rating of 5.6 and a blend of genres, "Goodfellas 2" is sure to provide an exciting and unforgettable cinematic experience.

<|user|>give me a newer one
<|user|>
give me the search query about the above conversation.
<|assistant|>
"Recommend a cool gangster movie with a blend of comedy and romance, released in more recent years than the original 'Goodfellas.'"
END
text:
<|system|>You are a helpful assistant.

Here are the movies you MUST choose from:

title: Goodfel